### Installations and Imports

In [1]:
!pip install tensorflow

In [2]:
!pip install --upgrade tensorflow

In [47]:
!pip install opencv-python

  Obtaining dependency information for opencv-python from https://files.pythonhosted.org/packages/38/d2/3e8c13ffc37ca5ebc6f382b242b44acb43eb489042e1728407ac3904e72f/opencv_python-4.8.1.78-cp37-abi3-win_amd64.whl.metadata
   ---------------------------------------- 0.0/38.1 MB ? eta -:--:--
   ---------------------------------------- 0.1/38.1 MB 2.0 MB/s eta 0:00:20
   ---------------------------------------- 0.2/38.1 MB 2.1 MB/s eta 0:00:19
   ---------------------------------------- 0.3/38.1 MB 2.4 MB/s eta 0:00:16
   ---------------------------------------- 0.4/38.1 MB 2.4 MB/s eta 0:00:16
    --------------------------------------- 0.6/38.1 MB 2.6 MB/s eta 0:00:15
    --------------------------------------- 0.7/38.1 MB 2.7 MB/s eta 0:00:14
    --------------------------------------- 0.9/38.1 MB 2.8 MB/s eta 0:00:14
   - -------------------------------------- 1.1/38.1 MB 2.9 MB/s eta 0:00:13
   - -------------------------------------- 1.2/38.1 MB 3.0 MB/s eta 0:00:13
   - -----------

In [95]:
import os
import numpy as np
import pandas as pd
import cv2 as cv

import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPool2D, BatchNormalization, Dropout

import matplotlib
import matplotlib.pyplot as plt

from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, Flatten

from keras.losses import BinaryCrossentropy
from keras.optimizers.legacy import Adam
from keras.optimizers import schedules

import xml.etree.ElementTree as ET
import matplotlib.image as mpimg
from matplotlib.patches import Rectangle
from matplotlib.image import imread
from PIL import Image

import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

#imports for mapping section of code 
import plotly.express as px

### Loading the Image Localization Model and Helper Functions

In [4]:
# USED THE CLASS FROM https://towardsdatascience.com/model-sub-classing-and-custom-training-loop-from-scratch-in-tensorflow-2-cc1d4f10fb4e
class SmokeDetection(tf.keras.models.Model):
    def __init__(self, givenModel, **kwargs):
        super().__init__(**kwargs)

        # Define Model
        self.model = givenModel

    def call(self, input, **kwargs):
        return self.model(input, **kwargs)

    def compile(self, optimizer, class_loss, local_loss, **kwargs):
        super().compile(**kwargs)

        self.local_loss = local_loss
        self.class_loss = class_loss
        self.optimizer = optimizer

    def train_step(self, batch, **kwargs):
        # imgs = batch[0]
        # gt_class_with_bbox = batch[1]
        # batch[1][0] = TRUE class and batch[1][1] is array of TRUE coordinates

        imgs, gt_class_with_bbox = batch

        # imgs, gt_class_with_bbox = self.imageAug(old_imgs, old_gt_class_with_bbox)

        with tf.GradientTape() as tape:
          # class_pred = self.model(imgs, training=True)[0]
          # bbox_pred = self.model(imgs, training=True)[1]

            class_pred, bbox_pred = self.model(imgs, training=True)

            class_loss = self.class_loss(gt_class_with_bbox[0], class_pred)

            gt_bbox = tf.cast(gt_class_with_bbox[1], tf.float32)

            local_loss = self.local_loss(gt_bbox, bbox_pred)

            # We want loss to worry more about localizing the smoke rather than classifying it
            # This follows from the fact that our images only belong to 1 class and the main problem
            # is detecting where exactly in the picture the smoke is
            tot_loss = class_loss * 0.5 + local_loss
            change = tape.gradient(tot_loss, self.model.trainable_variables)

        optimizer.apply_gradients(zip(change, self.model.trainable_variables))

        return {"total_loss": tot_loss, "class_loss": class_loss, "local_loss": local_loss}

    # THESE NEXT TWO FUNCTIONS ARE OURS
    def imageAug(self, imgs, cl_with_bbox):

        new_imgs = []
        new_cl_with_bbox = []

        for indx in range(2):
            temp_bbox = [BoundingBox(x1=cl_with_bbox[indx][0], y1=cl_with_bbox[indx][1], x2=cl_with_bbox[indx][2], y2=cl_with_bbox[indx][3], label=cl_with_bbox[indx][0])]
            temp_img = [imgs]

            bb = BoundingBoxesOnImage(temp_bbox, shape=(244, 244))

            aug_img, aug_bb = data_augment(images=temp_img, bounding_boxes=bb)
            new_bbox = [[0], [aug_bb[0].x1, aug_bb[0].y1, aug_bb[0].x2, aug_bb[0].y2]]

            new_imgs.append(aug_img)
            cl_with_bbox.append(new_bbox)

        return new_imgs, cl_with_bbox
    
    # Added test_step that basically also calculates the loss function of the Validation Data
    # Validation data is how we can interpret when the model has finished learning or how well it is really doing
    def test_step(self, validation_data):
        valid_data = validation_data

        val_imgs, val_classes_with_bbox = valid_data

        val_class_pred, val_bbox_pred = self.model(val_imgs, training=False)
        val_class_loss = self.class_loss(val_classes_with_bbox[0], val_class_pred)

        val_gt_bbox = tf.cast(val_classes_with_bbox[1], tf.float32)
        val_local_loss = self.local_loss(val_gt_bbox, val_bbox_pred)

        val_tot_loss = val_class_loss * 0.5 + val_local_loss

        return {"total_loss": val_tot_loss, "class_loss": val_class_loss, "local_loss": val_local_loss}

In [5]:
# USED basic outline of model code from website above but MODIFIED the model architecture through lots of experimentation with models
def create_model():
    # Define an input layer that takes in size (244, 244, 3) where 3 is number of channels - RGB
    input = Input(shape=(244, 244, 3))
    
    # We use the pre-trained model VGG16 with its weights
    base_model = VGG16(weights = 'imagenet', include_top = False, input_shape = (244, 244, 3))
    
    # MODIFIED: We froze every layer but the last convolutional block to keep the features that the model has learned
    for layer in base_model.layers[:-5]:
        layer.trainable = False
    
    # MODIFIED ARCHITECTURE
    
    # This is the model for classification. We just have one fully connected Dense layer with 256 units
    # Dropout is added to prevent the model from fullying relying on a select few number of neurons in the layer
    class_out = GlobalAveragePooling2D()(base_model.output)
    class_out = Dense(units=256)(class_out)
    class_out = Dropout(0.2)(class_out)
    
    # Sigmoid activation function in the end because this is a binary value either 0 or 1
    class_out = Dense(units=1, activation='sigmoid')(class_out)
    
    # This is the model for localization. We have 3 fully connected Dense layers with different number of units
    local_out = GlobalAveragePooling2D()(base_model.output)
    local_out = Dense(units=2048)(local_out)
    local_out = Dense(units=1024)(local_out)
    local_out = Dropout(0.2)(local_out)
    local_out = Dense(units=512)(local_out)
    local_out = Dropout(0.3)(local_out)
    
    # Relu activation function between we did not normalize bounding box values so they can be larger than 1 
    local_out = Dense(units=4, activation='relu')(local_out)
    
    # Combine models into one model 
    model = Model(inputs = base_model.input, outputs = [class_out, local_out])
    
    return model

In [34]:
# We need to resize the images to fit the pre-trained model we are going to use
def resizeImages(imgs, new_width, new_height):
    
    resizedImages = []
    
    # Just resizes
    for indx in range(len(imgs)):
        resizedImages.append(imgs[indx].resize((new_width, new_height)))
    
    return resizedImages

# We normalize the pixel values because it is more computationally efficient to deal with numbers between 0-1 rather than 0-255
def normalize(imgs):
    
    normalized = []
    
    # Loop through each img, convert to array of pixel values, and divide each by 255
    for img in imgs:
        img_arr = np.array(img)
        img_arr = img_arr / 255
        normalized.append(img_arr)
        
    return normalized

# If we scale the images, we have to scale the bounding boxes now as well otherwise it wont localize the smoke anymore
def scaleBoundingBoxes(bboxes, new_width, new_height, prev_width, prev_height):
    
    scaled_bboxes = []
    
    for indx in range(len(all_files)):
        
        # Get the scaling factors
        x_Scale_Factor = new_width / prev_width
        y_Scale_Factor = new_height / prev_height
        
        # Get the new bottom left corner and top right corner 
        xmin = bboxes[0] * x_Scale_Factor
        ymin = bboxes[1] * y_Scale_Factor
        xmax = bboxes[2] * x_Scale_Factor
        ymax = bboxes[3] * y_Scale_Factor
        
        scaled_bboxes.append([xmin, ymin, xmax, ymax])
    
    return scaled_bboxes

# In order to understand image localization, we need a function to draw the bounding boxes on the image
# We have a default value for predicted bounding box because sometimes we just want to draw the ground truth bounding box only
def drawBoundingBoxes(img, gt_bbox, pred_bbox = []):

    gt_boxWidth = gt_bbox[2] - gt_bbox[0]
    gt_boxHeight = gt_bbox[3] - gt_bbox[1]
    
    # We might not always have a predicted box if we havent ran predictions yet. So we need to check for that case
    if len(pred_bbox) != 0:
        pred_boxWidth = pred_bbox[2] - pred_bbox[0]
        pred_boxHeight = pred_bbox[3] - pred_bbox[1]
    
    # Show the image
    plt.imshow(img)
    
    # Plot the rectangle or bounding box that localizes the smoke
    gt_rectangle = Rectangle((gt_bbox[0], gt_bbox[1]), gt_boxWidth, gt_boxHeight, facecolor='none', edgecolor='black')
    plt.gca().add_patch(gt_rectangle)
    
    # If we have a predicted box then we also display what the prediction looks like as well
    if len(pred_bbox) != 0:
        pred_rectangle = Rectangle((pred_bbox[0], pred_bbox[1]), pred_boxWidth, pred_boxHeight, facecolor='none', edgecolor='red')
        plt.gca().add_patch(pred_rectangle)
    

def predict(model, test_img):
    pred_img = np.expand_dims(test_img, 0)
    pred = model.predict(pred_img)
    
    return pred

In [36]:
# Custom function that we wrote however the math and idea of IoU is not ours
def IoU(model, test_imgs, test_bboxes):
    """
    This function will find the mean of the IoU of the specified model on the test images
    """
    iou = 0
    
    for indx in range(len(test_imgs)):
        
        resized = resizeImages([test_imgs[indx]], 244, 244)
        normalized = normalize(resized)
        
        pred = predict(model, normalized[0])
        
        pred = scaleBoundingBoxes(pred[1][0], 640, 480, 244, 244)[0]
        
        # Calculate the area of the ground truth box
        gt_width = test_bboxes[indx][2] - test_bboxes[indx][0]
        gt_height = test_bboxes[indx][3] - test_bboxes[indx][1]
        
        gt_area = gt_width * gt_height
        
        # Calculate the area of the prediction box
        pred_width = pred[2] - pred[0]
        pred_height = pred[3] - pred[1]
        
        pred_area = pred_width * pred_height
        
        # Calculates intersection of the boxes
        x_intersect_min = max(test_bboxes[indx][0], pred[0])
        y_intersect_min = max(test_bboxes[indx][1], pred[1])
        x_intersect_max = min(test_bboxes[indx][2], pred[2])
        y_intersect_max = min(test_bboxes[indx][3], pred[3])
        
        # IoU cant be negative so we need to make sure area is not lower than 0
        intersect_width = max(0, x_intersect_max - x_intersect_min)
        intersect_height = max(0, y_intersect_max - y_intersect_min)
        
        intersection = intersect_width * intersect_height
        
        # Calculate the union area
        union = gt_area + pred_area - intersection
        
        iou += (intersection / union)
    
    iou /= len(test_imgs)
    
    return iou

In [7]:
best_model = SmokeDetection(create_model())

In [8]:
best_model.load_weights('Models/smokeModelWithDropout')

### Pre-load Images and Bounding Boxes

In [31]:
df = pd.read_csv('SmokeDataset.csv')
img_dir = 'day_time_wildfire_v2/images/'
all_files = df.to_numpy()

# Function to get all of the images in the directory
def getImagesFromDirectory(img_dir, all_files):
    
    imgs = []
    
    # Loops through all of the images in the directory, opens them, and adds them to the list
    for indx in range(len(all_files)):
        tempImage = Image.open(os.path.join(img_dir, all_files[indx, 0]) + '.jpeg')
        
        imgs.append(tempImage)
        
    return imgs

# Gets all of the classes and bounding boxes for each image
def getLabelsFromDirectory(all_files):
    
    classes = []
    bboxes = []
    
    # Reads the csv file that we made and gets the bounding boxes and class of each image (they should all be smoke labeled)
    for indx in range(len(all_files)):
         
        # Getting bottom left corner and top right corner to define the bounding box
        xmin = all_files[indx, 4]
        ymin = all_files[indx, 5]
        xmax = all_files[indx, 6]
        ymax = all_files[indx, 7]
        
        bboxes.append([xmin, ymin, xmax, ymax])
        
        # If it is smoke labeled we put a 0 and else we put a 1
        class_name = all_files[indx, 3]
        if class_name == 'smoke':
            classes.append(0)
        else:
            classes.append(1)
        
    return classes, bboxes

images = getImagesFromDirectory(img_dir, all_files)
classes, bboxes = getLabelsFromDirectory(all_files)

### Loading Forest Fire Area Predictor Model

In [73]:
fires = pd.read_csv('wildfiredb_2016_subset.csv')

columns = ['ELEV_mean', 'SLP_mean', 'EVT_mean', 'EVH_mean', 'TEMP_min', 'TEMP_max', 'PRCP', 'WSPD_ave', 'PRES_ave']
X = fires[columns]  # Independent variables

# Apply a logarithm transform to make the data more normalized (initially skewed with many values close to 0)
fires['frp_scaled'] = np.log10(fires['frp'] + 1)
Y = fires['frp_scaled']  # Dependent variable

# Separate 80% data into training set and 20% into test set
# random_state=0 ensures that the results are reproducible
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.20, random_state=0)

# Applying k-nearest neighbors model
RF_model = RandomForestRegressor(n_estimators = 50, max_depth = 30, max_samples=5000)
RF_model.fit(X_train, Y_train)

KNeighborsRegressor(n_neighbors=20)

### Load California Fire Incidents Data

In [98]:
# Just loads data that is in the California Fire jupyter notebook and gets ready to use it
df = pd.read_csv(r"California_Fire_Incidents.csv") 
df = df.drop_duplicates()
df = df.drop(columns = ['CanonicalUrl','Location','UniqueId','ConditionStatement','ControlStatement', 'SearchDescription','SearchKeywords'])

### Menu Functions 

In [45]:
def displayBasicMenu():
    print("Welcome to the Forest Fire Visualizer!")
    
    print("\nMenu")
    print("0. Quit")
    print("1. Get Statistics about a specific California County")
    print("2. View Fire Radiative Power Predictor Model")
    print("3. View Smoke Detection Model ")
    
    user_input = int(input("\nPlease enter the number beside the menu selection that you would like to view: "))
    
    return user_input

def displayAreaPredictorMenu():
    print("Welcome to our Forest Fire Radiative Power Predictor!")
    
    print("\nOptions")
    print("0. Quit")
    print("1. Get Information on the Parameters that our Model takes in")
    print("2. View a Correlation Heatmap of the Parameters that our Model takes in")
    print("3. Predict Fire Radiative Power")
    
    user_input = int(input("\nPlease enter the number beside the menu selection that you would like to view: "))
    
    return user_input

def displaySmokeDetectionMenu():
    print("Welcome to the our Smoke Detection Model!")
    
    print("\nOptions")
    print("0. Quit")
    print("1. Get Information on how the Model Works")
    print("2. Show Predictions on a Specific Image in the Dataset (Requires Filename)")
    print("3. Show Predictions on a 9 Random Images in the Dataset")
    print("4. Get the IoU Accuracy on a Specific Range of Images in the Dataset")
    print("5. Simulate Real-Time Detection by Applying the Model to a Video")
    
    user_input = int(input("\nPlease enter the number beside the menu selection that you would like to view: "))
    
    return user_input

In [93]:
def get_statistics_county(county_name):
    print("")
    county_data = df[df['Counties'] == county_name]

    if county_data.empty:
        return f"No data found for {county_name}."

    total_incidents = county_data.shape[0]
    total_acres_burned = county_data['AcresBurned'].sum()
    average_acres_burned = county_data['AcresBurned'].mean()

    return f"Statistics for {county_name}:\nTotal Incidents: {total_incidents}\nTotal Acres Burned: {total_acres_burned}\nAverage Acres Burned per Incident: {average_acres_burned:.2f}"

def frpPredictorInformation():
    print("\n\nThe following information is from the WildfireDB Page.")
    print("Our model takes in the following parameters from WildfireDB. Each value refers to one 375 by 375 meter squared area in the U.S.\n")
    
    # Information from https://wildfire-modeling.github.io/
    print('ELEV_mean: Mean elevation over the observed area from 2016.')
    print('SLP_mean: Mean slope over the observed area from 2016')
    print('EVT_mean: Mean existing vegetation type.')
    print('EVH_mean: Mean existing vegetation height.')
    print('TEMP_min: Minimum temperature.')
    print('TEMP_max: Maximum temoerature.')
    print('PRCP: Precipitation.')
    print('WSPD_ave: Average relative wind speed.')
    print('PRES_ave: Average atmospheric pressure.')

    print('\nFire Radiative Power: The model predicts the fire radiative power in Watts given the above vegetation and weather conditions.\n\n')

def viewHeatMap():
    heatmap = sns.heatmap(fires[cols].corr()[['frp']].sort_values(by='frp', ascending=False), vmin=-1, vmax=1, annot=True)
    heatmap.set_title('Correlations Between Parameters and Fire Radiative Power (FRP)');

def predictFRP():
    print("Please enter the following values as floating point values (decimals): \n")
    
    # Takes in Weather and Vegetation Data
    elev_mean = float(input("Mean Elevation: "))
    slp_mean = float(input("Mean Slope: "))
    evt_mean = float(input("Mean Vegetation Type: "))
    evh_mean = float(input("Mean Vegetation Height: "))
    temp_min = float(input("Minimum Temperature: "))
    temp_max = float(input("Maximum Temperature: "))
    prcp = float(input("Precipitation: "))
    wspd_ave = float(input("Average relative wind speed: "))
    pres_ave = float(input("Average atmospheric pressure: "))
    
    frp_predict = RF_model.predict([[elev_mean, slp_mean, evt_mean, evh_mean, temp_min, temp_max, prcp, wspd_ave, pres_ave]])
    
    # Print the predicted FRP after reversing the log transform that was applied
    print(f"Predicted Fire Radiative Power: {10**(frp_predict[0]-1)} Watts.")

def smokeDetectionInformation():
    print("\n\nOur custom smoke detection model is an image localization model that predicts where the smoke in an image is.")
    
    print("Our model utilizes transfer learning by utilizing VGG16 as the pre-trained base model. We chose VGG16 because")
    print("it is lightweight compared to many other pre-trained models. It only has 16 layers with weights which reduces the large")
    print("number of parameters that other models have. Additionally, VGG16 has been trained on over 1000 different classes and therefore")
    print("has learned many high level features that we wanted to use for our purpose. On top of the base model we added our own custom")
    print("architecture for classification (determining if smoke is in the image or not by giving a probability) and localization")
    print("(coordinates of the bounding box that highlights the smoke in the image). The model was trained on 1533 images with 438")
    print("images for cross-validaton and tested on 219 images.\n\n")
    
    
def predictOnFile():
    file = input("Please enter JUST the file name (no path) the image in the dataset: ")
    
    print("Black represents the labeled bounding box")
    print("Red represents the predicted bounding box")
    
    for indx in range(len(all_files)):
        if all_files[indx, 0] == file:
            # Image manipulation so it fits model input
            resized = resizeImages([images[indx]], 244, 244)
            normalized = normalize(resized)
            
            # Predict
            pred = predict(best_model, normalized[0])
            
            # Because our images are actually 640 x 480 we want to scale the bounding box prediction and image back to normal
            scaled_pred = scaleBoundingBoxes(pred[1][0], 640, 480, 244, 244)
            
            # Draw the Box
            drawBoundingBoxes(images[indx], bboxes[indx], scaled_pred[0])
            
            return
    
    print("Could not find file")

def predictOnRandom():
    # Let's try to visualize multiple different predictions using the NoDropout Model
    plt.figure(figsize=(18, 15))

    for subplot in range(9):
        plt.subplot(3, 3, subplot + 1)

        # Get a random index from the test images
        image_indx = np.random.randint(0, len(images))
        
        resized = resizeImages([images[image_indx]], 244, 244)
        normalized = normalize(resized)
        
        # predict on the test image
        pred = predict(best_model, normalized[0])
        
        scaled_pred = scaleBoundingBoxes(pred[1][0], 640, 480, 244, 244)
        
        # pred[0] has the classification probability while pred[1][0] is the array of bounding box
        drawBoundingBoxes(images[image_indx], bboxes[image_indx], scaled_pred[0])

def getIoUAccuracy():
    
    beg_input = int(input("Enter the beginning endpoint for the range of images you want to test on (0-2189)"))
    end_input = int(input("Enter the ending endpoint for the range of images you want to test on (0-2189)\n"))
    
    while(beg_input >= end_input):
        print("First number cannot be bigger than Second!!!")
        beg_input = int(input("Enter the beginning endpoint for the range of images you want to test on (0-2189)"))
        end_input = int(input("Enter the ending endpoint for the range of images you want to test on (0-2189)\n"))
    
    while(not(0 <= beg_input <= 2189) or not(0 <= end_input <= 2189)):
        print("Numbers must be IN RANGE!!!")
        beg_input = int(input("Enter the beginning endpoint for the range of images you want to test on (0-2189)"))
        end_input = int(input("Enter the ending endpoint for the range of images you want to test on (0-2189)\n"))
    
    print(IoU(best_model, images[beg_input:end_input], bboxes[beg_input:end_input]))

# https://youtube.com/watch?v=AxIc-vGaHQ0 Used this video to learn 
def applyToVideo():
    cap = cv.VideoCapture('InputVideo/20160604-FIRE-smer-tcs3-mobo-c.mp4')
    width = int(cap.get(cv.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv.CAP_PROP_FRAME_HEIGHT))
    
    # What we are writing new video to
    output = cv.VideoWriter('OutputVideo/Output_20160604-FIRE-smer-tcs3-mobo-c.mp4', cv.VideoWriter_fourcc(*'MP4V'), 20.0, (width, height))
    
    xScaleFactor = width / 244
    yScaleFactor = height / 244
    
    while cap.isOpened():
        ret, frame = cap.read()
        
        if not ret:
            break
        
        img = frame
        img = cv.resize(img, (244, 244))
        img = img / 255
        
        pred = predict(best_model, img)
        
        xmin = int(pred[1][0][0] * xScaleFactor)
        ymin = int(pred[1][0][1] * yScaleFactor)
        xmax = int(pred[1][0][2] * xScaleFactor)
        ymax = int(pred[1][0][3] * yScaleFactor)
        
        cv.rectangle(frame, (xmin, ymin), (xmax, ymax), (0, 0, 255), 2)
        
        output.write(frame)
        if(cv.waitKey(10)) & 0xFF == ord('q'):
            break
    
    cap.release()
    output.release()
    cv.destroyAllWindows()
    
    print("You can now view the real-time detection video in the Ouput Video Folder")

## INTERACTIVE MENU TO USE

In [94]:
while(True):
    user_input = displayBasicMenu()
    
    next_input = -1
    
    if user_input == 0:
        break
    elif user_input == 1:
        next_input = input("Please enter the name of the county: ")
        
        stats = get_statistics_county(next_input)
        print(stats)
        
    elif user_input == 2:
        next_input = displayAreaPredictorMenu()
        
        if next_input == 0:
            break
        elif next_input == 1:
            frpPredictorInformation()
        elif next_input == 2:
            viewHeatMap()
            break
        elif next_input == 3:
            predictFRP()
        
    elif user_input == 3:
        next_input = displaySmokeDetectionMenu()
        
        if next_input == 0:
            break
        elif next_input == 1:
            smokeDetectionInformation()
        elif next_input == 2:
            predictOnFile()
            break
        elif next_input == 3:
            predictOnRandom()
            break
        elif next_input == 4:
            getIoUAccuracy()
        elif next_input == 5:
            applyToVideo()

Welcome to the Forest Fire Visualizer!

Menu
0. Quit
1. Get Statistics about a specific California County
2. View Forest Fire Area Predictor Model
3. View Smoke Detection Model 

Please enter the number beside the menu selection that you would like to view: 1
Please enter the name of the county: Monterey

Statistics for Monterey:
Total Incidents: 45
Total Acres Burned: 156566.0
Average Acres Burned per Incident: 3479.24
Welcome to the Forest Fire Visualizer!

Menu
0. Quit
1. Get Statistics about a specific California County
2. View Forest Fire Area Predictor Model
3. View Smoke Detection Model 

Please enter the number beside the menu selection that you would like to view: 0
